In [ ]:
# Machine Learning model to predict whether financial statements indicate a <6% Tier 1 Capital Ratio

1. See what columns are essential
    * Load in first two columns of every file
    * def load_one(path):
    Read the file (tab-delimited, skip the description row)

    df = pd.read_csv(
   
        path,
        sep="\t",
        header=0,
        nrows=2,
        dtype=str
    )
    * Look at descriptions and how many files have what
3. Determine essential columns to keep and notate
4. Any files that don't have essential columns are dropped
5. Read in all files, merge to single DataFrame
6. Force essential columns to float32
    * df[c] = pd.to_numeric(df[c], errors="coerce").astype(np.float32) for c in essential_cols
    * Some models force conversion to float64, sklearn LogisticRegression does this
7. Train, Test, Split
8. Predict

FFIEC Website: https://cdr.ffiec.gov/public/ManageFacsimiles.aspx

Bulk Data: https://cdr.ffiec.gov/public/PWS/DownloadBulkData.aspx

Silent trip ups: 

- Have non-proportional amounts of what you're classifying.

- Not having enought samples

In [8]:
import pandas as pd
import glob

In [11]:
path = r'/RCRI Schedules'
ffiec_files = glob.glob(path + "/*.txt")

In [14]:
# Read each txt file into DataFrame
# This creates a list of dataframes
df_list = pd.read_csv(file) for file in ffiec_files

SyntaxError: invalid syntax (1401780041.py, line 3)

In [15]:
df_list

<generator object <genexpr> at 0x000002350593CD60>

In [ ]:
# Concatenate all DataFrames
big_df = pd.concat(df_list, ignore_index=True)

In [7]:
df_rc = pd.read_csv(
    "RCRI Schedules\FFIEC CDR Call Schedule RCRI 12312024.txt",
    sep="\t",
    header=0,      # Use the first line as headers (the RCFD codes)
    low_memory=False # Reads whole file into memory first, determiens d-type second. No mixed column d-types.
)
df_rc

,IDRSSD,RCFA2170,RCFA3128,RCFA3792,RCFA5310,RCFA5311,RCFA7204,RCFA7205,RCFA7206,RCFA8274,...,RCOAP866,RCOAP867,RCOAP868,RCOAP870,RCOAP872,RCOAP875,RCOAQ258,RCOAS540,RCOWH312,Unnamed: 154
0,NaN,TOTAL ASSETS,ALLOCATED TRANSFER RISK RESERVES,TOTAL RISK-BASED CAPITAL,AACL INCLUDIBLE IN TIER2 CAPITAL,TIER 2 (SUPPLEMENTARY) CAPITAL,TIER 1 LEVERAGE CAPITAL RATIO,TOTAL RISK-BASED CAPITAL RATIO,TIER 1 RISK-BASED CAPITAL RATIO,TIER 1 CPTL ALLWBL UNDR RISK-BASED,...,TIER 2 CAP INSTRMNTS PLUS RLTD SRPLS,NNQLIFY CAP INSTR PHSOUT TIER2 CAP,CAP MNRTY INTRST NOT INCL TIER 1 CAP,TIER 2 CAPITAL BEFORE DEDUCTIONS,TOTAL TIER 2 CAPITAL DEDUCTIONS,DEDS COMEQTY TIER1 CAP ADD TIER1 CAP,CUMULATIVE CHANGE IN FAIR VALUE,UNCONDITIONALLY OR OTHER AMOUNT,TOTAL APPLICABLE CAPITAL BUFFER,NaN
1,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN
2,242.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN
3,279.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,132,0,0,NaN,NaN
4,354.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,587,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4539,5879201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,186,0,0,0,NaN,NaN,NaN
4540,5887420.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4541,5903517.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,105,NaN,NaN
4542,5972661.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [ ]:
def load_one(path: Path) -> pd.DataFrame:
    df = pd.read_csv(
        path,
        sep="\t",
        header=0,       # codes in row 0
        low_memory=False,
        dtype=str,      # read as string first to avoid mixed-type surprises
    )

F1-Score - a machine learning evaluation metric that measures a classifier's accuracy by taking the harmonic mean of precision and recall. It's a particularly useful metric for evaluating models, especially with imbalanced datasets, because it balances the trade-off between high precision and high recall. A perfect F1 score is 1 (100%), while the worst score is 0.
https://www.google.com/search?client=firefox-b-1-d&q=machine+learning+f1+score

Mean Absolute Error - Average distance of all data points from fitted trend line

Validation Accuracy (For Random Forest) - How often model is correct overall. Not as useful if you don't have 50/50 split of classes (or 25/25/25/25 or whatever)

In [ ]:
# Read in one file. 
# List columns and descriptions
# List any file that does not contain that exact same columns

In [ ]:
# Need more quarters worth of data, get at least 50 < 6%s. Just download as many as possible and look at
# Look at all columns for each file
# Any that are exactly the same, great
# If not, do they have what we need?
# Check dtypes